In [ ]:
# Transformers installation
! pip install transformers datasets
! pip install seqeval
! pip install evaluate
! pip install wandb

import numpy as np

# Preprocess Data


## Load Tokenizer and Dataset

Inspec Dataset has 3 datasets: train, test, and validation

### Data Types

*   train: [*sample*]
*   *sample*: {
  document: [string],
  doc_bio_tags: [int]
  }

In [ ]:
from datasets import load_dataset

inspec = load_dataset("midas/inspec")

Repo card metadata block was not found. Setting CardData to empty.


In [ ]:
example = inspec["train"][0]

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

## Preprocess

We need to


*   Convert words to tokens
*   Realign the tokens and labels, since a single word may correspond to multiple tokens.
*   Ignore the special tokens [CLS] and [SEP] in the loss function.
*   Only label the first token of a given word, and assign -100 to the others.
*   Convert doc_bio_tags to integers for classificatioon



In [ ]:
label_list = np.unique(example["doc_bio_tags"])

id2label = {i: label for i, label in enumerate(label_list)}
label2id = {v: k for k, v in id2label.items()}

print('Mapping doc_bio_tag to integer:\n\n',label2id)
print('\nMapping integer to doc_bio_tag:\n\n',id2label)

Mapping doc_bio_tag to integer:

 {'B': 0, 'I': 1, 'O': 2}

Mapping integer to doc_bio_tag:

 {0: 'B', 1: 'I', 2: 'O'}


In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["document"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"doc_bio_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label2id[label[word_idx]]) # Convert BIO to integers for classification
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
# This is our tokenized dataset (a data_dict which we will convert to a TF Dataset)
tokenized_inspec = inspec.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

## Utils for Training

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, return_tensors="tf")

In [ ]:
import evaluate

seqeval = evaluate.load("seqeval")

In [ ]:
labels = example[f"doc_bio_tags"]


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    log_results = {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }
    wandb.log(log_results)

    return log_results

## Experiment Tracking with W&B

In [ ]:
import wandb

wandb.login()
wandb.init(project="ppp-keyword-extraction")

wandb: Currently logged in as: warrensw (ac215-ppp). Use `wandb login --relogin` to force relogin


## Train with multi-GPU
Reference: https://saturncloud.io/docs/examples/python/tensorflow/qs-multi-gpu-tensorflow/

In [ ]:
import tensorflow as tf
import keras
import time
from transformers import TFAutoModelForTokenClassification
from transformers.keras_callbacks import KerasMetricCallback
from transformers import create_optimizer

def train_multigpu(n_epochs, base_lr, batchsize):
    num_train_steps = (len(tokenized_inspec["train"]) // batchsize) * n_epochs

    # Set up for multi-GPU training
    strategy = tf.distribute.MirroredStrategy()
    print("Number of devices: %d" % strategy.num_replicas_in_sync)

    with strategy.scope():
        model = TFAutoModelForTokenClassification.from_pretrained(
                  "distilbert-base-uncased", num_labels=3, id2label=id2label, label2id=label2id)

        # We define our own optimizer (and lr_schedule which we do not use)
        optimizer, lr_schedule = create_optimizer(
                                                  init_lr=2e-5,
                                                  num_train_steps=num_train_steps,
                                                  weight_decay_rate=0.01,
                                                  num_warmup_steps=0,
                                                 )
        # The model is ready for training
        model.compile(optimizer=optimizer, metrics=["accuracy"])

    # Load in our data as TF Datasets (with data_collator applied)
    train_ds = model.prepare_tf_dataset(
                                        tokenized_inspec["train"],
                                        shuffle=True,
                                        batch_size=batchsize,
                                        collate_fn=data_collator,
                                       ).prefetch(2).cache().shuffle(1000)

    valid_ds = model.prepare_tf_dataset(
                                        tokenized_inspec["validation"],
                                        shuffle=False,
                                        batch_size=batchsize,
                                        collate_fn=data_collator,
                                       ).prefetch(2)

    # Set up a callback for metrics at the end of every epoch
    metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=valid_ds)
    callbacks = [metric_callback]

    # Run training
    start = time.time()

    model.fit(x=train_ds, validation_data=valid_ds, epochs=n_epochs, callbacks=callbacks)

    end = time.time() - start
    print("model training time", end)

    # Save the trained model
    path = "model/keras_multi/"
    tf.keras.models.save_model(model, path)
    print(f"Saved model to {path}")

In [ ]:
# Call the training function with specified parameters
model_params = {
    "n_epochs": 3,
    "base_lr": 2e-5,
    "batchsize": 16
}

tester_plain = train_multigpu(**model_params)

Number of devices: 1


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForTokenClassification: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertForTokenClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForTokenClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForTokenClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able t

Epoch 1/3
 6/62 [=>............................] - ETA: 35s - loss: 0.7634 - accuracy: 0.3013

62/62 [==============================] - 93s 1s/step - loss: 0.3689 - accuracy: 0.8864 - val_loss: 0.2480 - val_accuracy: 0.3638 - precision: 0.2105 - recall: 0.1937 - f1: 0.2017
Epoch 2/3
62/62 [==============================] - 59s 945ms/step - loss: 0.2303 - accuracy: 0.8946 - val_loss: 0.2243 - val_accuracy: 0.3672 - precision: 0.3401 - recall: 0.3678 - f1: 0.3534
Epoch 3/3
62/62 [==============================] - 59s 957ms/step - loss: 0.2115 - accuracy: 0.8972 - val_loss: 0.2193 - val_accuracy: 0.3682 - precision: 0.3637 - recall: 0.4111 - f1: 0.3860
model training time 234.77801513671875


## Test dataset (ignore for now)

In [ ]:
from transformers import pipeline

from transformers.modeling_tf_utils import load_tf_weights

num_train_steps = (len(tokenized_inspec["train"]) // model_params["batchsize"]) * model_params["n_epochs"]

optimizer, lr_schedule = create_optimizer(
                                          init_lr=2e-5,
                                          num_train_steps=num_train_steps,
                                          weight_decay_rate=0.01,
                                          num_warmup_steps=0,
                                         )
custom_objects = {
            "optimizer": optimizer
}

model = tf.keras.models.load_model("model/keras_multi/", custom_objects)

text = "The Golden State Warriors are an American professional basketball team based in San Francisco."
classifier = pipeline("ner", model=model)
classifier(text)